In [1]:
#https://medium.com/saarthi-ai/sentence-classification-using-convolutional-neural-networks-ddad72c7048c

In [2]:
import pandas as pd
path = '/Users/ivowings/Desktop/sentiment labelled sentences'

filepath_dict = {'yelp': '/yelp_labelled.txt' ,
                 'amazon': '/amazon_cells_labelled.txt',
                 'imdb': '/imdb_labelled.txt'}
df_list = []
for source, filepath in filepath_dict.items():    
    df = pd.read_csv(path+filepath, names=['sentence', 'label'], sep='\t')
    # Add another column filled with the source name
    df['source'] = source 
    df_list.append(df)
df = pd.concat(df_list)
print(df.head())

                                            sentence  label source
0                           Wow... Loved this place.      1   yelp
1                                 Crust is not good.      0   yelp
2          Not tasty and the texture was just nasty.      0   yelp
3  Stopped by during the late May bank holiday of...      1   yelp
4  The selection on the menu was great and so wer...      1   yelp


In [1]:
from sklearn.model_selection import train_test_split             
from keras.preprocessing.text import Tokenizer                    
from keras.preprocessing.sequence import pad_sequences
df_yelp = df[df['source'] == 'yelp']

sentences = df_yelp['sentence'].values
y = df_yelp['label'].values

sentences_train,sentences_test,y_train,y_test = train_test_split(
                                                sentences, y,  
                                                test_size=0.25,  
                                                random_state=1000)

tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(sentences_train)

X_train = tokenizer.texts_to_sequences(sentences_train)
X_test = tokenizer.texts_to_sequences(sentences_test)
# Adding 1 because of  reserved 0 index
vocab_size = len(tokenizer.word_index) + 1                          

maxlen = 100

X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

NameError: name 'df' is not defined

In [ ]:
X_train

In [5]:
import numpy as np

def create_embedding_matrix(filepath, word_index, embedding_dim):
    vocab_size = len(word_index) + 1  
    # Adding again 1 because of reserved 0 index
    embedding_matrix = np.zeros((vocab_size, embedding_dim))

    with open(filepath) as f:
        for line in f:
            word, *vector = line.split()
            if word in word_index:
                idx = word_index[word] 
                embedding_matrix[idx] = np.array(vector, dtype=np.float32)[:embedding_dim]

    return embedding_matrix

In [8]:
embedding_dim = 50
embedding_matrix = create_embedding_matrix('/Users/ivowings/Desktop/sentiment labelled sentences/glove.6B.50d.txt' ,
                                            tokenizer.word_index,  
                                            embedding_dim)

In [17]:
vocab_size

1747

In [14]:
from keras.models import Sequential
from keras import layers
embedding_dim = 100

model = Sequential()
model.add(layers.Embedding(vocab_size, embedding_dim, input_length=maxlen))
model.add(layers.Conv1D(128, 5, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
history = model.fit(X_train, y_train,
                    epochs=10,
                    validation_data=(X_test, y_test),
                    batch_size=10)

Epoch 1/10
75/75 [==============================] - 2s 13ms/step - loss: 0.6913 - accuracy: 0.4819 - val_loss: 0.6752 - val_accuracy: 0.5680
Epoch 2/10
75/75 [==============================] - 1s 9ms/step - loss: 0.5872 - accuracy: 0.7243 - val_loss: 0.5325 - val_accuracy: 0.7560
Epoch 3/10
75/75 [==============================] - 1s 10ms/step - loss: 0.2497 - accuracy: 0.9642 - val_loss: 0.4733 - val_accuracy: 0.8120
Epoch 4/10
75/75 [==============================] - 1s 11ms/step - loss: 0.0541 - accuracy: 0.9912 - val_loss: 0.5420 - val_accuracy: 0.7960
Epoch 5/10
75/75 [==============================] - 1s 10ms/step - loss: 0.0124 - accuracy: 0.9998 - val_loss: 0.5461 - val_accuracy: 0.8040
Epoch 6/10
75/75 [==============================] - 1s 10ms/step - loss: 0.0063 - accuracy: 1.0000 - val_loss: 0.5862 - val_accuracy: 0.8080
Epoch 7/10
75/75 [==============================] - 1s 10ms/step - loss: 0.0027 - accuracy: 1.0000 - val_loss: 0.6157 - val_accuracy: 0.8120
Epoch 8/10
75/